### LangChain Agent Setup for YouTube QA Bot
This notebook creates a LangChain agent that can answer questions based on the Eleo German learning video transcript using FAISS and OpenAI.


In [7]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langsmith import traceable 

In [8]:

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Load FAISS vector store
db = FAISS.load_local(
    folder_path="../data/vectorstores/eleo_faiss",
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True
)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=retriever
)

###  Wrap QA Chain as a LangChain Tool


In [9]:
def answer_question_only_result(query: str) -> str:
    return qa_chain.invoke({"query": query})["result"]

qa_tool = Tool(
    name="VideoQA",
    func=answer_question_only_result,
    description="Answers questions about the Eleo German learning video"
)

### Create Zero-Shot Agent (No Memory)


In [10]:
# Set up the LLM again
llm = ChatOpenAI(model="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# Create the agent with your tool
agent = initialize_agent(
    tools=[qa_tool],
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_54723/949731264.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_54723/949731264.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react

### Run the Agent for a Sample Question


In [11]:
@traceable(name="Eleo_QA_Session")  # this labels the function for LangSmith
def ask_agent_traced(question: str):
    ask_agent_traced("What does Eleo say about pronunciation?") # New way with LangSmith tracking



### Add Memory to the Agent (ConversationBufferMemory)


In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

### Create Conversational Agent (With Memory)


In [13]:
from langchain.agents import initialize_agent, AgentType

agent_with_memory = initialize_agent(
    tools=[qa_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

### Run a Multi-Turn Conversation with Memory


In [14]:
# Ask the agent a question
response_1 = agent_with_memory.run("How does Eleo recommend practicing German pronunciation?")
print(response_1)

# Follow-up question
response_2 = agent_with_memory.run("Can you explain that again more simply?")
print(response_2)

/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_54723/1332888517.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_1 = agent_with_memory.run("How does Eleo recommend practicing German pronunciation?")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: VideoQA
Action Input: How does Eleo recommend practicing German pronunciation?
Observation: Eleo recommends practicing German pronunciation by engaging in self-talk in the language you are learning. This involves having conversations with yourself in German during daily activities. Additionally, listening to and speaking with native speakers or other German speakers is ideal. If you don't have someone to practice with, watching videos with authentic content, like those on FluentU, can also help improve your pronunciation and vocabulary.
Thought:Do I need to use a tool? No
AI: Eleo recommends practicing German pronunciation by engaging in self-talk in the language you are learning, having conversations with yourself in German during daily activities, listening to and speaking with native speakers or other German speakers, and watching videos with authentic content.

> Finished chain.
Eleo recommends pr

### Restricts answers to content

In [15]:
from langchain.prompts import PromptTemplate

# Custom prompt template
custom_prompt = PromptTemplate.from_template(
    template="""
You are an assistant that answers questions only using the following transcript chunk.
If the answer is not contained within it, reply with "I don't know."

Transcript:
{context}

Question: {question}
Answer:
"""
)

### Create the RetrievalQA with the Custom Prompt

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=False
)